<a href="https://colab.research.google.com/github/rodrigosarloaf/projeto_DL_23_1/blob/main/junta_prodes_deter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Pacotes
import pathlib
from osgeo import ogr, gdal, gdalconst
from skimage.morphology import disk, dilation, erosion
import os
import numpy as np
import shutil
from google.colab import drive
import cv2
from datetime import date, timedelta
import pandas as pd
import pickle

In [2]:
# Conectando ao drive
drive.mount('/content/drive', force_remount=True)
root_path = '/content/drive/My Drive/projeto_DL/'
os.chdir(root_path)

Mounted at /content/drive


In [3]:
# Juntando as séries de PRODES e DETER para serem usadas nos modelos
# Veja que o PRODES termina em 30/07 - é a data de referência para taxas de desmatamento

# Tratamendo de dados mensais
# Puxando csv de referência
df_mensal = pd.read_csv(
    '/content/drive/My Drive/projeto_DL/deter_tratado/referencia_mes.csv'
    ).drop(columns = 'Unnamed: 0')
df_mensal.head()

,begin,end,patch_name
0,2015/01/01,2015/02/01,0
1,2015/02/01,2015/03/01,1
2,2015/03/01,2015/04/01,2
3,2015/04/01,2015/05/01,3
4,2015/05/01,2015/06/01,4


In [4]:
# Jogando fora as primeiras linhas que são do "ano-PRODES" anterior ao começo da nossa análise
df_mensal = df_mensal.loc[df_mensal.begin >= '2015/08/01']
# Jogando fora as últimas linhas que são do "ano-PRODES" posterior ao fim da nossa análise
df_mensal = df_mensal.loc[df_mensal.begin < '2021/08/01']
#
df_mensal = df_mensal.reset_index(drop=True)
df_mensal = df_mensal.sort_values(by = ['begin'])

In [5]:
path_mes = '/content/drive/My Drive/projeto_DL/deter_tratado/mensal/'
path_prodes = "/content/drive/My Drive/projeto_DL/prodes_tratado/"

contador = 2016
for i in range(0, len(df_mensal), 12):
  # Filtrando df interno para operarmos nele
  df_interno = df_mensal.loc[i:(i+11)].copy()
  df_interno = df_interno.reset_index(drop=True)
  # Lendo o PRODES adequado
  with open(path_prodes+'prodes_'+str(contador)+'.pkl', 'rb') as handle:
    prodes_atual = pickle.load(handle)
  
  for j in range(len(df_interno)):
    # Abrindo o DETER
    imagem_deter = cv2.imread(path_mes + str(df_interno['patch_name'][j]) + '.tif', cv2.IMREAD_GRAYSCALE)
    imagem_deter = imagem_deter.astype('float64')
    # Acrescentando os alertas ao PRODES
    prodes_atual = np.add(prodes_atual, imagem_deter)

    if(np.sum(prodes_atual[prodes_atual > 1])):
      prodes_atual[prodes_atual > 1] = 1
    
    # Salvando
    with open("/content/drive/My Drive/projeto_DL/dataset_tratado/mes/"+str(contador)+"_"+str(j+1)+".pkl", 'wb') as handle:
      pickle.dump(prodes_atual, handle, protocol=pickle.HIGHEST_PROTOCOL)
  
  contador = contador+1
  print("i = " + str(i))


i = 0
i = 12
i = 24
i = 36
i = 48
i = 60


In [6]:
# Checando
import time
path_checar = "/content/drive/My Drive/projeto_DL/dataset_tratado/mes/"

for i in os.listdir(path_checar):
  with open(path_checar+i, 'rb') as handle:
    arq_checar = pickle.load(handle)
  print(i)
  print(np.unique(arq_checar, return_counts=True))
  time.sleep(5)


2016_1.pkl
(array([0., 1.]), array([5009272, 1544328]))
2016_2.pkl
(array([0., 1.]), array([5009272, 1544328]))
2016_3.pkl
(array([0., 1.]), array([5009272, 1544328]))
2016_4.pkl
(array([0., 1.]), array([5009272, 1544328]))
2016_5.pkl
(array([0., 1.]), array([5009272, 1544328]))
2016_6.pkl
(array([0., 1.]), array([5009272, 1544328]))
2016_7.pkl
(array([0., 1.]), array([5009272, 1544328]))
2016_8.pkl
(array([0., 1.]), array([5009272, 1544328]))
2016_9.pkl
(array([0., 1.]), array([5009272, 1544328]))
2016_10.pkl
(array([0., 1.]), array([5009272, 1544328]))
2016_11.pkl
(array([0., 1.]), array([5009272, 1544328]))
2016_12.pkl
(array([0., 1.]), array([5009272, 1544328]))
2017_1.pkl
(array([0., 1.]), array([4864637, 1688963]))
2017_2.pkl
(array([0., 1.]), array([4843379, 1710221]))
2017_3.pkl
(array([0., 1.]), array([4833312, 1720288]))
2017_4.pkl
(array([0., 1.]), array([4828727, 1724873]))
2017_5.pkl
(array([0., 1.]), array([4828727, 1724873]))
2017_6.pkl
(array([0., 1.]), array([4827878, 

In [ ]:
# Vou ficar apenas com os dados mensais por enquanto. Estou acabando com o armazenamento do drive